# PrefixSpan
PrefixSpan ist ein weiterer Algorithmus zum Auffinden häufiger Sequenzen in Transaktionsdatenbanken. Er extrahiert Sequenzmuster durch rekursives Vergrößern von Präfixen und versucht so das wiederholte, aufwendige Generieren und Prüfen von Kandidatenmengen zu vermeiden.

In [ ]:
from tui_dsmt.fpm import SequentialItemset, dna

## Inhaltsverzeichnis
- [Der Datensatz](#Der-Datensatz)
- [Finden von einelementigen Sequenzen](#Finden-von-einelementigen-Sequenzen)
- [Unterteilung des Suchraums](#Unterteilung-des-Suchraums)
- [Sequenzen in Partitionen](#Sequenzen-in-Partitionen)
- [Der finale Algorithmus](#Der-finale-Algorithmus)

## Der Datensatz
Im Folgenden wird der Algorithmus anhand eines Beispiels veranschaulicht. Verwendet wird erneut die kleine Transaktionsdatenbank mit einigen fiktiven DNA-Sequenzen.

In [ ]:
dna

Der Support wird wie bereits beim GSP-Algorithmus im Voraus festgelegt.

In [ ]:
min_supp = 3

## Finden von einelementigen Sequenzen
Der erste Schritt ist erneut das Suchen von häufigen, einelementigen Sequenzen. In der Datenbank werden dazu - wie bereits mehrfach zuvor - alle Transaktionen gescannt, die 1-Itemsets gezählt und nach dem vorgebenen, minimalen Support gefiltert.

Die nachfolgende Funktion kommt Ihnen deshalb vermutlich aus dem vorangegangenen Notebook bekannt vor:

In [ ]:
def itemsets1(transactions, min_supp):
    # leere Abbildung anlegen
    C1 = {}

    # alle Transaktionen durchgehen
    for _, items in transactions:
        # alle Elemente der Transaktion betrachten
        for element in items:
            # Element zur Abbildung hinzufügen
            # bzw. Anzahl um 1 erhöhen
            if element not in C1:
                C1[element] = 1
            else:
                C1[element] += 1

    # Elemente nach minimalem Support filtern
    # und in Itemset umwandeln
    return set(SequentialItemset(item) for item, count in C1.items()
               if count >= min_supp)


F1 = itemsets1(dna, min_supp)
F1

## Unterteilung des Suchraums
Anhand der verbliebenen Sequenzen der Länge $1$ (nur `C` wurde auf Grund des minimalen Supports entfernt) wird der Suchraum für die weiteren Schritte partitioniert. Die Items dienen dabei als Präfix, sodass in der ersten Partition alle verbliebenen Optionen mit dem Präfix `A` untersucht werden sollen, in der zweiten Partition alle Optionen mit dem Präfix `G` und in der letzten Partition alle mit Präfix `T`.

Die erste Transaktionsdatenbank besteht bisher aus allen verfügbaren (in diesem Fall $3$) unbeschnittenen DNA Sequenzen. Für die Partitionierung werden aus dieser sogenannte **projizierte Datenbanken** mit einem Präfix nach folgendem Schema hergeleitet:
- Transaktionen, die das Präfix nicht enthalten, werden nicht in die projizierte Datenbank aufgenommen.
- Transaktionen, welche das Präfix enthalten, werden ohne das Präfix übernommen.
- Das erste vollständige Auftauchen des Präfix wird gewertet.

Nach diesem Prinzip lassen sich zunächst einzelne Transaktionen projizieren:
$$
<A, C, B, C, A, D> \; \xrightarrow{A} \; <C, B, C, A, D>
$$$$
<A, C, B, C, A, D> \; \xrightarrow{C} \; <B, C, A, D>
$$$$
<A, C, B, C, A, D> \; \xrightarrow{C, B} \; <C, A, D>
$$

Durch Hintereinanderausführung dieser Projektionen verlängert sich einfach das verwendete Präfix. Es ist daher nicht relevant, ob mit einem langen Präfix oder mit mehreren kurzen projiziert wird, sofern die Reihenfolge übereinstimmt:
$$
<A, C, B, C, A, D> \; \xrightarrow{C} \; <B, C, A, D> \; \xrightarrow{B} \; <C, A, D>
$$

Eine prozijierte Datenbank ist nun eine Transaktionsdatenbank, für die jede Transaktion nach dem gegebenen Präfix projiziert wurde. Im folgenden Beispiel wird die projizierte DNA-Datenbank für das Präfix `A` erzeugt:

In [ ]:
D_A = dna.project('A')
D_A

## Sequenzen in Partitionen
Jede der entstandenen projizierten Datenbanken wird nun einzeln verarbeitet, indem innerhalb der verbliebenen Teiltransaktionen erneut nach häufigen, den minimalen Support überschreitenden Sequenzen der Länge $1$ gesucht wird. Diese Zählung schließt durch die zuvor erfolgte Projektion das verwendete Präfix implizit ein.

In [ ]:
itemsets1(D_A, min_supp)

Ausgehend von den gefundenen, häufigen Sequenzen der Länge $1$ unter Einbezug des angegebenen Präfix, werden erneut projizierte Datenbanken abgeleitet. Von diesem ausgehend wird der Algorithmus rekursiv fortgesetzt, bis in einem Schritt die projizierte Datenbank leer ist.

In [ ]:
D_AG = D_A.project('G')
D_AG

Das Paar leerer Klammern bedeutet, dass das Präfix in der Transaktion enthalten war, jedoch das Ende markierte und demnach eine leere Sequenz verbleibt.

## Der finale Algorithmus
Der Algorithmus setzt die gezeigten Einzelschritte nun rekursiv zusammen.

In [ ]:
def rec(D, min_supp, prefix=()):
    # Häufige Sequenzen der Länge 1 finden.
    frequent_sequences = itemsets1(D, min_supp)

    # Jede häufige Sequenz der Länge 1 einzeln betrachten.
    for sequence in frequent_sequences:
        # Präfix mit häufiger Sequenz als (Zwischen-) Ergebnis zurückgeben.
        yield prefix + sequence

        # Datenbank projizieren, neues Präfix bilden, rekursiver Aufruf
        projected_database = D.project(*sequence)
        new_prefix = prefix + sequence
        yield from rec(projected_database, min_supp, new_prefix)


# Aufruf mit vollständiger Datenbank, min_supp und leerem Präfix
set(rec(dna, min_supp))